# SDM With Combination

### Step 1: Import everything and load variables

In [1]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./iranthology-dataset-tira')


Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


I will use a small hardcoded example located in ./iranthology-dataset-tira.
The output directory is /tmp/


### Step 2: Load the Data

In [9]:
queries = pt.io.read_topics(input_directory + '/milestone-1-topics.xml', format='trecxml')

#qrels = open(input_directory + '/qrels.txt', "r")

documents = [json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r')]
documents = [{'docno': i['docno'], 'text': i['text'], 'title': i['original_document']['title'], 'abstract': i['original_document']['abstract']} for i in documents]

print('We look at the first document:\n')
print(documents[0])

Step 2: Load the data.
We look at the first document:

{'docno': '2019.sigirconf_workshop-2019birndl.0', 'text': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019 ', 'title': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019', 'abstract': ''}


### Step 3: Create the Index

In [4]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100, 'title': 10240, 'abstract': 10240, 'text': 10240}, blocks=True)
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.


 31%|█████████████▌                              | 16558/53673 [00:35<00:18, 1984.37it/s]

11:54:02.329 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


100%|█████████████████████████████████████████████| 53673/53673 [01:06<00:00, 811.33it/s]


11:54:57.297 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


### Retrieval Pipelines 

### TF-IDF
Term Frequency - Inverse Document Frequency is a metric, which tells which term is relevant for a certain document. The metric counts the frequency of the term compared to in how many documents it is. If the term is frequent in a document but not as frequent in as many documents, it can be seen that this term is a key word for this particular document.

### BM25

Is a bag-of-words (topic modeling) approach to information retrieval. BM25 ranks the documents based on the terms that are in the document. We noticed during the Milestone 2, that BM25 was very efficient.

In [48]:
tf_idf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

### Step 4: Create Retrieval Pipeline

Originally we wanted to try neural scoring systems like Bert out, which would have a lot more context-based approach to ranking. The notebook + docker kinda restricted us, which is the reason why we decided against it. SDM (Sequential Dependence Model) expands the query by connecting words that appear together / in the same context, which is the reason why we thought it could be more accurate. We try SDM query expansion together with different scoring systems.

In [ ]:
sdm = pt.rewrite.SequentialDependence()

The query is first expandend with sdm. After that 100 top results ranked with a linear combination of bm25 and tf_idf are retrieved. We wanted to combine these to get the term frequency and the topic modeling approaches together. Linear combiantion gives bigger results, which is the reason why we added the weights.

In [136]:
retrieval_pipeline = sdm >> (0.5*bm25 + 0.5*tf_idf)%100

### Step 5: Create the run

In [137]:
run = retrieval_pipeline(queries)
run.head()

,qid,docid,docno,score,query,query_0,rank
0,1,14568.0,2019.wsdm_conference-2019.40,23.553173,relevant documents show bigram fake news synon...,relevant documents show the bigram of fake ne...,0
1,1,7566.0,2008.sigirconf_conference-2008.116,22.540051,relevant documents show bigram fake news synon...,relevant documents show the bigram of fake ne...,1
2,1,21754.0,2020.cikm_conference-2020.118,22.374995,relevant documents show bigram fake news synon...,relevant documents show the bigram of fake ne...,2
3,1,33491.0,2018.wwwconf_conference-2018c.140,21.483913,relevant documents show bigram fake news synon...,relevant documents show the bigram of fake ne...,3
4,1,14643.0,2019.wsdm_conference-2019.115,20.639545,relevant documents show bigram fake news synon...,relevant documents show the bigram of fake ne...,4


### Step 6: Persist Run

In [135]:
print('Step 6: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='multi-field', depth=100)

Step 6: Persist Run.


### Self-Reflection

#### Iris

A lot of time of the Milestone 3 was I used for trying to configure docker and the things that I needed to be able to work with pyterrier. In the end everything worked out, but I used an awful lot of time, which left less time for the actual pyterrier. It was difficult to figure out ways to rank the documents, especially because there wasn't really much information except the pyterrier documentation. I still found it a lot of trying out different approaches and learning about them.

### Justin
The tasks for milestone 3 were a bit more fun for me because we could try different approaches to find a better retrieval pipeline. Unfortunality we had problems with understanding the tutorial and which dataset we should use and whether we have to register it in TIRA. Then we thought we can evaluate our approaches directly in TIRA but we wondered why we get no results instantly. As we know that the evaluating made manually we could concentrate on find approaches. But we didn't find out how we can evaluate our approaches correctly.